<a href="https://colab.research.google.com/github/bhatsbharath/generative_ai_agents/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatbot / Standalone Agent

Defines chat_with_ollama to send a prompt to a local Ollama model API (llama3) and return the generated response. Uses the requests library to make a POST request to http://localhost:11434/api/generate. If successful, it returns the generated text; otherwise, an error message.

In [ ]:
import requests

def chat_with_ollama(prompt, model="llama3"):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False  # stream = True if you want to process partial responses
    }

    response = requests.post(url, json=payload)
    if response.status_code == 200:
        return response.json()['response']
    else:
        return f"Error: {response.status_code} - {response.text}"

# Example usage
response = chat_with_ollama("What is multi agent system in generative ai.")
print(response)


This code defines a function generate_response that sends a prompt to a local language model API (llama3) and gets a generated text response.

It uses the requests library to POST the prompt with parameters like model and temperature.

If successful, it returns the generated text; otherwise, it shows an error.

The script tests two temperature values (0.2 and 1.2) to show how temperature affects creativity:

Lower temperature → more predictable responses.

Higher temperature → more creative and varied responses.

In [ ]:
import requests

def generate_response(prompt, model="llama3", temperature=0.7):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "temperature": temperature,
        "stream": False
    }

    response = requests.post(url, json=payload)
    if response.status_code == 200:
        return response.json()["response"]
    else:
        return f"Error: {response.status_code} - {response.text}"


prompt = "Write a short sci-fi story in 4-5 sentences about a robot that becomes self-aware.."

# Try different temperatures
for temp in [0.2, 1.2]:
    print(f"\n--- Temperature: {temp} ---")
    print(generate_response(prompt, temperature=temp))




# Vision Language Model

In [ ]:
import ollama

response = ollama.chat(
    model='llama3.2-vision',
    messages=[{
        'role': 'user',
        'content': 'What is in this image?',
        'images': ['TestImage.png']
    }]
)

print(response)

# Prompting

1. Zero-Shot Prompting

The model is asked to perform a task without any examples. It must rely solely on its prior knowledge.

In [ ]:
# Zero-Shot Prompting
prompt = "Translate 'I like pizza' into French."
print(generate_response(prompt))

2. One-Shot Prompting

The model is given one example of the task before being asked to perform it.

In [ ]:
# One-Shot Prompting
prompt = """Translate the following sentences into French:

English: Good morning
French: Bonjour

English: I like pizza
French:"""
print(generate_response(prompt))


3. Few-Shot Prompting

Few-Shot prompting gives the model multiple examples of a task before asking it to complete a similar one. This helps the model better understand the pattern and produce more accurate results.

In [ ]:
#Few shot prompting
prompt = """Translate the following sentences into French:

English: Good morning
French: Bonjour

English: How are you?
French: Comment ça va ?

English: I like pizza
French:"""
print(generate_response(prompt))


4. Chain-of-Thought Prompting

This technique encourages the model to think step-by-step by including reasoning or intermediate steps in the prompt, improving performance on complex tasks. It’s often done by adding explanations or guiding the model through a thought process.

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts import PromptTemplate

llm = ChatOllama(
	model="llama3.2",
	temperature=0.7,
	top_p=0.9,
	num_predict=256,
	repeat_penalty=1.1
)

# Standard prompt
standard_prompt = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question concisely: {question}."
)

# Chain of Thought prompt
cot_prompt = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question step by step concisely: {question}"
)

# Create chains
standard_chain = standard_prompt | llm
cot_chain = cot_prompt | llm

# Example question
question = "If a train travels 120 km in 2 hours, what is its average speed in km/h?"

# Get responses
standard_response = standard_chain.invoke(question).content
cot_response = cot_chain.invoke(question).content

print("Standard Response:")
print(standard_response)
print("\nChain of Thought Response:")
print(cot_response)